In [1]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import nltk
import re 
import pickle
import gensim


In [2]:
SF = pd.read_csv('SF_jobs_new.csv')
SF.drop('Unnamed: 0',axis=1,inplace=True)
NYC = pd.read_csv('NYC_jobs_new.csv')
NYC.drop('Unnamed: 0',axis=1,inplace=True)
Seattle = pd.read_csv('Seattle_jobs_new.csv')
Seattle.drop('Unnamed: 0',axis=1,inplace=True)

In [3]:
def line_cleaner(line): # perform tokenization, select noun, Lemmertization etc on a line text
    rtext=[]    
    for w, tag in nltk.pos_tag(nltk.word_tokenize(line.lower())):  # Tokenization
        if tag in ['NN']:  # Keep only Nouns (for this special case)
            rtext.append(w)
    #text = re.sub("[^a-zA-Z+3#]"," ", line)
    
    #rtext = text.lower().split()
    
    wordnet_lemmatizer = WordNetLemmatizer()  # Lemmertization
    rtext = [wordnet_lemmatizer.lemmatize(w) for w in rtext]
            
    stop_words = set(stopwords.words("english") + ['e','g','etc','data','customer','experience','business','+','requirement','dr','ad','hoc','f','eco','ecosystem','pl','schema','nice']) # Filter out any stop words
    rtext = [w for w in rtext if not w in stop_words]
    
    return rtext # return list of tokens

def text_cleaner(html,Oneline=True): # return a single line of cleaned text

    soup_obj = BeautifulSoup(html,"lxml") # Get the html from the site

    for script in soup_obj(["script", "style"]):
        script.extract() # Remove these two elements from the BS4 object
    
    text = soup_obj.body.get_text('\n') # Get the text from this

    lines = (line.strip() for line in text.splitlines()) # break into lines

    chunks = (phrase.strip() for line in lines for phrase in line.split("  ")) # break multi-headlines into a line each
    
    def chunk_space(chunk):
        chunk_out = chunk + ' ' # Need to fix spacing issue
        return chunk_out  

    text = ''.join(chunk_space(chunk) for chunk in chunks if chunk).encode('utf-8') # Get rid of all blank lines and ends of line

    # Now clean out all of the unicode junk (this line works great!!!)
    try:
        text = text.decode('unicode_escape').encode('ascii', 'ignore') # Need this as some websites aren't formatted
    except:                                                            # in a way that this works, can occasionally throw
        return                                                         # an exception

    text = re.sub("[^a-zA-Z+3#]"," ", text)  # Now get rid of any terms that aren't words (include 3 for d3.js)
      
    text = line_cleaner(text)
    
    return text

In [4]:
cleanDescription = []
for i in range(SF.shape[0]):
    cleanDescription.append(text_cleaner(SF.job_description[i]))
for i in range(NYC.shape[0]):
    cleanDescription.append(text_cleaner(NYC.job_description[i]))
for i in range(Seattle.shape[0]):
    cleanDescription.append(text_cleaner(Seattle.job_description[i]))

In [5]:
with open('cleaned_doc.txt','wb') as file1:
    pickle.dump(cleanDescription,file1,protocol=2)

In [2]:
cleaneddoc = pd.read_pickle('cleaned_doc.txt')


In [15]:
model = gensim.models.Word2Vec(cleaneddoc, size=20, window=6, min_count=5, workers=2)
model.save('W2Vmodel')

In [3]:
model = gensim.models.Word2Vec.load('W2Vmodel')
model.similarity('mapreduce','communication')

-0.043460787788028053

In [4]:
model.most_similar('python')

[('c++', 0.9234117865562439),
 ('ruby', 0.9224433302879333),
 ('scala', 0.9091888070106506),
 ('language', 0.8705078363418579),
 ('programming', 0.8683212399482727),
 ('java', 0.845077633857727),
 ('perl', 0.8362725973129272),
 ('backend', 0.832685649394989),
 (u'algorithm', 0.8098684549331665),
 ('django', 0.8036363124847412)]

In [6]:
model.most_similar('hadoop')

[('spark', 0.9294838905334473),
 ('mapreduce', 0.8779422044754028),
 ('kafka', 0.8647394180297852),
 ('cloudera', 0.8630594611167908),
 ('pig', 0.8551802635192871),
 ('cassandra', 0.8516056537628174),
 ('hbase', 0.8491491079330444),
 ('nosql', 0.8358124494552612),
 ('aws', 0.8078849911689758),
 ('hdfs', 0.8070412278175354)]

In [10]:
model.similarity('r','html')

0.1913360517297627